### Treinamento do Modelo Machine Learning

In [25]:
%pip install pandas scikit-learn xgboost joblib


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Importações de Bibliotecas

In [27]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, accuracy_score
from typing import List, Tuple
from joblib import dump

#### Treinamento e Predições do Modelo XGBoost

In [29]:
# Carregar os dados do arquivo CSV
# TODO: Precisamos buscar os dados no S3
data = pd.read_csv("../notebooks/datasets/btc_dataset.csv", index_col=0)

# Definir preditores
# Estas são as features que estamos usando para prever o target
predictors: List[str] = [
    "close",
    "volume",
    "open",
    "high",
    "low",
    "edit_count",
    "sentiment",
    "neg_sentiment",
]

# Função de predição
def predict(train: pd.DataFrame, test: pd.DataFrame, predictors: List[str], model: XGBClassifier) -> pd.DataFrame:
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="predictions")
    return pd.concat([test["target"], preds], axis=1)

# Função de backtesting
# Usaremos uma abordagem de janela móvel para fazer o backtesting do modelo.
# Pegaremos uma janela de dados, usaremos para treinar o modelo e, em seguida, usaremos o modelo para prever a próxima janela de dados.
# Repetiremos esse processo até termos usado todos os dados.
# O parâmetro start significa 3 anos de dados para treinar o modelo.
# O parâmetro step significa 150 dias de dados para testar o modelo.
def backtest(data: pd.DataFrame, model: XGBClassifier, predictors: List[str], start: int = 1095, step: int = 150) -> pd.DataFrame:
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i + step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)

    return pd.concat(all_predictions)

# Função de avaliação do modelo
def evaluate_model(predictions: pd.DataFrame) -> Tuple[float, float]:
    precision = precision_score(predictions["target"], predictions["predictions"])
    accuracy = accuracy_score(predictions["target"], predictions["predictions"])
    return precision, accuracy

# Criar e treinar o modelo XGBoost
model = XGBClassifier(random_state=1, learning_rate=0.1, n_estimators=200)
predictions = backtest(data, model, predictors)

# Avaliar o modelo
precision, accuracy = evaluate_model(predictions)
print(f"Precisão do modelo: {precision:.2f}")
print(f"Acurácia do modelo: {accuracy:.2f}")
print(predictions)

Precisão do modelo: 0.50
Acurácia do modelo: 0.49
                           target  predictions
2017-09-16 00:00:00+00:00       0            0
2017-09-17 00:00:00+00:00       1            1
2017-09-18 00:00:00+00:00       0            1
2017-09-19 00:00:00+00:00       0            1
2017-09-20 00:00:00+00:00       0            1
...                           ...          ...
2024-09-01 00:00:00+00:00       1            0
2024-09-02 00:00:00+00:00       0            0
2024-09-03 00:00:00+00:00       1            0
2024-09-04 00:00:00+00:00       0            0
2024-09-05 00:00:00+00:00       0            1

[2547 rows x 2 columns]


#### Serialização do Modelo

In [28]:
# Salvar o modelo treinado em um arquivo
model_filename = "btc_trend_prediction_model.joblib"
dump(model, model_filename)

['btc_trend_prediction_model.joblib']